In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

In [22]:
cotton_price = 1.5
nitrogen_cost = 0.9
nitrogen_ratio = nitrogen_cost/cotton_price
sq_rate = 100
sq_yield = 1500
yield_sd = 100
yield_noise = 5
eonr_sd = 25
rst_b2g = -0.02
opt_pcts = [50, 75, 100]
rate_rngs = [25, 50, 100]

In [23]:
nitrogen_ratio

0.6

In [3]:
trial = np.load('../data/Trial_Design.npy')
trial_names = np.load('../data/Trial_Design_names.npy', allow_pickle=True)
trial_idx = [i for i, n in enumerate(trial_names) if not 'rep' in n]
trial_coords = np.load('../data/Trial_Design_coords.npy')
trial_coords_idx = np.int0(np.floor(trial_coords/3))
n = trial_coords.shape[0]

rst_sim = np.load('../data/Trial_sim.npy')
b0_true = rst_sim[:100]
b1_true = rst_sim[100:]

meig = np.load('../data/Trial_meig.npy').transpose([-1, -2, -3])

In [32]:
# avg_eonr = sq_rate - 6
# rst_yield_ref =  rst_b0 + rst_b1 * sq_rate + rst_b2g * sq_rate**2
# rst_yield_opt_f =  rst_b0 + rst_b1 * avg_eonr + rst_b2g * avg_eonr**2

# rst_net_ref = rst_yield_ref * cotton_price - sq_rate * nitrogen_cost
# rst_net_opt_f = rst_yield_opt_f * cotton_price - avg_eonr * nitrogen_cost 
# rst_net_f = (rst_net_opt_f - rst_net_ref).mean((1,2))
# rst_net_f

In [21]:
resultsdf = []
for opt_pct in opt_pcts:    
    rst_optr = opt_pct + eonr_sd * b1_true   
    rst_optr = np.clip(rst_optr, 0, 200)
    avg_eonr = rst_optr.mean((1,2))[:,None,None]
    

    rst_b1 = -2 * rst_b2g * rst_optr + nitrogen_ratio
    rst_b0 = sq_yield + yield_sd * b0_true
    rst_b0 = rst_b0 - (rst_b1 * sq_rate + rst_b2g * sq_rate **2) 
    
    rst_yield_ref =  rst_b0 + rst_b1 * sq_rate + rst_b2g * sq_rate**2
    rst_yield_opt_f =  rst_b0 + rst_b1 * avg_eonr + rst_b2g * avg_eonr**2
    rst_yield_opt =  rst_b0 + rst_b1 * rst_optr + rst_b2g * rst_optr**2

    rst_net_ref = rst_yield_ref * cotton_price - sq_rate * nitrogen_cost
    rst_net_opt_f = rst_yield_opt_f * cotton_price - avg_eonr * nitrogen_cost 
    rst_net_opt = rst_yield_opt * cotton_price - rst_optr * nitrogen_cost 

    rst_net_f = (rst_net_opt_f - rst_net_ref).mean((1,2))
    rst_net_max = (rst_net_opt - rst_net_ref).mean((1,2))


    for idx in tqdm(trial_idx):
        exp_name = trial_names[idx]
        trial_rst = np.nan_to_num(trial[idx])/2

        for rate_rng in rate_rngs:
            
            trial_rate = rate_rng * trial_rst
            inp_rate = trial_rate + sq_rate
            yield_rnd = np.random.randn(*rst_b0.shape) * yield_noise
            rst_yield_obs =  yield_rnd + rst_b0 + rst_b1 * trial_rate

            X = trial_rate.reshape(1, -1)
            Xc = meig.reshape(len(meig), -1)
            Xi = (Xc * X)
            Xp = (X ** np.array([0,1])[:,None])
            Xf = np.concatenate([Xp, Xc, Xi]).T
            y = rst_yield_obs.reshape(len(rst_yield_obs), -1).T
            xpred, _, _, _ = np.linalg.lstsq(Xf, y, rcond=None)

            b0_pred, b1_pred = xpred[2:].reshape(2, -1, xpred.shape[-1])
            b0_pred = (xpred[[0]] + np.dot(Xc.T, b0_pred)).T.reshape(rst_b0.shape)
            b1_pred = (xpred[[1]] + np.dot(Xc.T, b1_pred)).T.reshape(rst_b1.shape)
            
            rst_optr_pred = (b1_pred - nitrogen_ratio) / (-2 * rst_b2g)
            rst_optr_pred = np.clip(rst_optr_pred, 0, 200)
            avg_eonr_pred = rst_optr_pred.mean((1,2))[:,None,None]
    
            rst_yield_obs =  rst_b0 + rst_b1 * inp_rate + rst_b2g * inp_rate**2
            rst_yield_pred_f =  rst_b0 + rst_b1 * avg_eonr_pred + rst_b2g * avg_eonr_pred**2
            rst_yield_pred =  rst_b0 + rst_b1 * rst_optr_pred + rst_b2g * rst_optr_pred**2

            rst_net_obs = rst_yield_obs * cotton_price - inp_rate * nitrogen_cost 
            rst_net_pred_f = rst_yield_pred_f * cotton_price - avg_eonr_pred * nitrogen_cost 
            rst_net_pred = rst_yield_pred * cotton_price - rst_optr_pred * nitrogen_cost 

            rst_net_loss = (rst_net_obs - rst_net_ref).mean((1,2))
            rst_net_diff = (rst_net_pred - rst_net_ref).mean((1,2))
            rst_net_diff_f = (rst_net_pred - rst_net_pred_f).mean((1,2))
            prob_nrate = (np.abs(rst_optr_pred - rst_optr) < 5).mean((1,2))


            rnp = np.stack([rst_net_max, avg_eonr_pred.flatten(), prob_nrate, rst_net_loss, rst_net_diff, rst_net_diff_f], -1)
            df = pd.DataFrame(rnp, columns=['Max', 'AVG', 'Prob', 'Loss', 'Total', 'VRT'])
            df['rate_rng'] = rate_rng
            df['opt_pct'] = opt_pct
            df['Trial'] = exp_name
            df['Rep'] = np.arange(len(df))
            resultsdf.append(df)

resultsdf = pd.concat(resultsdf).reset_index(drop=True)
resultsdf.to_csv('../data/trial_spatial_linear.csv', index = False)